Assignment 5, Part 1: Using the file capitals_lat_lon.csv, form a data frame and write out an HTML file which when rendered in a browser displays a marker for each capital city.  Hint: use folium.

In [14]:
import pandas as pd
import folium
import random

# Load the data from the CSV file
df = pd.read_csv('capitals_lat_lon_csv_2.csv')

# Define a list of possible colors
colors = ['blue', 'darkblue', 'cadetblue','lightblue']

# Create a map centered around the average latitude and longitude
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
my_map = folium.Map(location=map_center, zoom_start=2)

# Iterate over the DataFrame and add markers to the map
for idx, row in df.iterrows():
    color = random.choice(colors)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Capital'],
        icon=folium.Icon(color=color)
    ).add_to(my_map)

# Save the map as an HTML file
my_map.save('capital_cities_map.html')

Assignment 5, Part 2: Estimate the latitude and longitude for three points of a triangle that would just cover Africa or come close.  (Hint: Rabat, Capetown, and Mogadishu, for example.)  Use folium to demonstrate where the points are on the map so that a person could see at a glance that they do.  Use Shapely to define a polygon from those three points.  Compute the area and perimeter of the triangle assuming flat earth and allowing each degree of latitude and each degree of longitude to be considered one unit of length.

In [22]:

from shapely.geometry import Polygon

# Define the three points
points = [
    {"name": "Rabat", "lat": 33.97159, "lon": -6.849813},
    {"name": "Gqeberha", "lat": -33.942586, "lon": 25.557316},
    {"name": "Qardho", "lat": 9.510417, "lon": 49.087270}
]



# Create a folium map centered around Africa
africa_center = [0, 20]
africa_map = folium.Map(location=africa_center, zoom_start=3)

# Add markers for each point
for point in points:
    folium.Marker(
        location=[point["lat"], point["lon"]],
        popup=point["name"]
    ).add_to(africa_map)

# Add lines to form the triangle
folium.PolyLine(
    locations=[[p["lat"], p["lon"]]
               for p in points] + [[points[0]["lat"], points[0]["lon"]]],
    color="blue"
).add_to(africa_map)

# Save the map to an HTML file
africa_map.save('africa_triangle_map.html')

# Define the polygon using Shapely
triangle = Polygon([(p["lon"], p["lat"]) for p in points])

# Compute the area and perimeter of the triangle
area = triangle.area
perimeter = triangle.length

# Print the area and perimeter
area, perimeter

(1503.1022554431454, 185.71643155786217)

Assignment 5, Part 3: Estimate the boundaries for Kansas (use just four lat-lon points) and Nebraska (use just six points).  Construct a geojson file from that.  Then write a Python program to read that file, form a dictionary, and plot the result.

In [5]:
import json
import folium

# Read the GeoJSON file
with open('kansas_nebraska.geojson', 'r') as file:
    data = json.load(file)

# Create a map centered around the middle of the USA
usa_center = [39.8283, -98.5795]
map = folium.Map(location=usa_center, zoom_start=5)

# Add the GeoJSON to the map
folium.GeoJson(data, name='geojson').add_to(map)

# Save the map as html
map.save('kansas_nebraska_map.html')


Assignment 5, Part 4: Generate at random the “happiness index” for each state in the USA.  Provide a choropleth of the USA with that data represented thereon.  Use Plotly and make sure the viewer can zoom in and out of the map and pan back and forth.

In [29]:
import pandas as pd
import numpy as np
import plotly.express as px

# List of US state abbreviations
state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
    'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH',
    'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Generate random happiness index for each state
np.random.seed(42) 
happiness_index = np.random.randint(0, 50, size=len(state_abbreviations))

# Create a DataFrame
df = pd.DataFrame({
    'State': state_abbreviations,
    'Happiness Index': happiness_index
})

# Create the choropleth map
fig = px.choropleth(
    df,
    locations='State',
    locationmode="USA-states",
    color='Happiness Index',
    scope="usa",
    color_continuous_scale="Viridis",
    labels={'Happiness Index': 'Happiness Index'},
    title='Random Happiness Index by State'
)
fig.update_layout(
    geo=dict(
        lakecolor='rgb(255, 255, 255)',
        projection_scale=1, 
        center=dict(lat=37.0902, lon=-95.7129)
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

# Show the map
fig.show()

Assignment 5, Part 5: Generate 500 points from the random exponential distribution.  Choose your own parameters for that.  Use Plotly Dash and plot a histogram of that distribution such that there are fifteen bins.  Try to make the plot interactive so the user can select the number of bins with a slider, pull down or text entry box.

In [36]:

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np

# Generate 500 points from a random exponential distribution
np.random.seed(42)
data = np.random.exponential(scale=2, size=500)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Graph(id='histogram'),
    html.Label('Number of Bins'),
    dcc.Slider(
        id='bin-slider',
        min=5,
        max=30,
        step=1,
        value=15,
        marks={i: str(i) for i in range(5, 31)}
    ),
    html.Div(id='slider-output-container')
])

# Define callback


@app.callback(
    Output('histogram', 'figure'),
    [Input('bin-slider', 'value')]
)
def update_histogram(num_bins):
    fig = px.histogram(data, x=data, nbins=num_bins,
                       title='Histogram of Random Exponential Distribution')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)